In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pyconll

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# !pip install stanza

In [ ]:
## Number of Sentences, words in the file


import os
import pyconll
import csv


def statistics(filepath):
  '''This function calculates the statistics of the corpus developed over phases.
      File path given to function has following file structure to access output CoNLL files:
      Filepath >> 
          Phases >> 
            Output files/temp_files/csv input files >>
              output CoNLL files 
  '''



  dirs = sorted(os.listdir(filepath))       # Creating sorted list of directories from given path

  result_list = []                          # Initializing list to add all statistics

  Total_sent = 0                      # Initializing variable to store total number of sentences in corpus
  Total_words = 0                     # Initializing variable to store total number of tokens in corpus
  Total_speakers = 0                  # Initializing variable to store total number of speakers in corpus
  Total_repair = 0                    # Initializing variable to store total number of repairs in corpus
  Total_disfluency =0                 # Initializing variable to store total number of disfluencies in corpus
  Total_hesitation = 0
  Total_codeswitch =0                 # Initializing variable to store total number of codeswitches in corpus
  Total_other_tags =0                 # Initializing variable to store total number of tags except repair, disfluencies and codeswitches in corpus
  Total_words_without_tags = 0        # Initializing variable to store total number of tokens except tagged in corpus
  Total_words_tags = 0                # Initializing variable to store total number of tagged tokens in corpus

  Total_time=0                      # Initializing variable to calculate total time of corpus dialect

  title = ['Phase','Filename','Duration','No.of_Speakers','No.of_Sentences','No.of_Tokens(all)','No.of_Repair',
           'No.of_Disfluency', 'Hesitation','No.of_Code-switching','No.of_OtherTags','No.of_Tokens(w/o chunks&OtherTags)']

  result_list.append(title)           # Adding heading to csv file

  # for every file in the directory
  for file in dirs:
      # print(file)
      if not os.path.isfile((filepath+str(file))):         # Ignoring the files and only considering folders in given path  

        phase_dir = sorted(os.listdir(filepath+str(file)))
        Phase = file[-1]                                  # Considering the phase of output files

        for phases in phase_dir:                           
          # print(phases)
          if not os.path.isfile((filepath+str(file)+"/"+str(phases))):
            outputfiles_dir = sorted(os.listdir(filepath+str(file)+"/"+str(phases)))   # Accessing output files folder in each phase

            for output_file in outputfiles_dir:
              
              filename = filepath+str(file)+"/"+str(phases)+"/"+str(output_file)    # Looping over each output file in output files folder

              if filename.endswith("_output.txt"):

                  filename_in = filename
    
                  file1 = pyconll.load_from_file(filename_in)        # Loading CoNLL output file using pyconll module


                  count_sent = 0             #Initializing variable for sentences in each output file
                  count_word = 0             #Initializing variable for tokens in each output file
                  count_tags = 0             #Initializing variable for tagged tokens in each output file
                  list_form= ["[laughter]", "[noise]", "[incomprehensible]", "[pause]","[aside]"]   # List of other tags than repair, disfluencies and codeswitch
                  list_form_count=0          #Initializing variable for tags in above list in each output file
                  count_repair = 0
                  count_disfluency = 0
                  count_hesitation = 0
                  count_codeswitch = 0
                  count_quote = 0
                  
                  # list_misc_count=0
                  misc_keys = []

                  for sentence in file1:
                      count_sent+=1             # Counting number of sentences in each output file
                      
                      for token in sentence:           
                          
                          if "_" in token.form:                           
                              chunk_split=token.form.split("_")           #Splitting token if it is chunk
          #                     print(chunk_split) 
                              count_word+=len(chunk_split)                #Counting words in the chunk 
                          else:
                              count_word+=1
                              

                          if token.form in list_form:
          #                     print(token.form)
                              list_form_count+=1           # Maintaining count of tags mentioned in above list

                          
                          for i in token.misc.keys():
                              if i=='Repair':
                                  count_repair+=1         # Counting repairs in output file
                              if i =='Disfluency':
                                  count_disfluency+=1     # Counting disfluencies in output file
                              
                              if i =='Hesitation':
                                  count_hesitation+=1

                              if i =='CodeSwitch':
                                  count_codeswitch+=1     # Counting codeswitch in output file  
                                  
                              if i =='Quote':
                                  count_quote+=1          # Counting quotes in output file
      
                 
                  total_chunks = count_repair + count_disfluency + count_codeswitch + count_quote
                  #     print("total_chunks: ", total_chunks)

                  tokens_without_tags = count_word - total_chunks - list_form_count   # Counting tokens without tags
                  
                  
                  end_time=0
                  speakers= []
                  
                  for sentence in file1:
                      speakers.append(sentence.meta_value('speaker_id'))
                      
                      if(sentence.meta_value('sent_id')==str(count_sent)):       # Storing end time of last sentence of output file to get total duration of file
                          end_time = (sentence.meta_value('end_time'))           
                          t = end_time.split(':')
                          total_min =int(t[1])
                      
                          if(float(t[2])>30.0):
                              total_min+=1
          
                  print(filename, set(speakers))                    
                  speakers = len(list(set(speakers)))          # Counting number of speakers in each output file
                  

                  Total_sent += count_sent          
                  Total_speakers += speakers
                  Total_words+= count_word
                  Total_repair +=count_repair
                  Total_disfluency +=count_disfluency
                  Total_hesitation +=count_hesitation
                  Total_other_tags +=list_form_count
                  Total_words_without_tags +=tokens_without_tags
                  Total_words_tags += count_word-list_form_count
                  Total_time +=total_min

                  

                  # print(tokens_without_tags)
                      
                  result_list.append([Phase,output_file.replace("_output", ""),end_time,speakers,
                                      count_sent,count_word,count_repair,count_disfluency,count_hesitation,
                                      count_codeswitch, list_form_count,tokens_without_tags])         # Appending stats of each output file
                  # import pdb;pdb.set_trace()
  result_list.append(["","Total Stats",str(Total_time)+" mins",Total_speakers,Total_sent, Total_words,Total_repair,Total_disfluency,Total_hesitation,
                    Total_codeswitch,Total_other_tags,Total_words_without_tags])        
           


  try:
      file_out = os.path.join(filepath, "Stat_20230516.csv")                           # Creating Stat.csv file in given filepath 
      with open(file_out, "w",encoding="utf-8", newline='',) as csvfile:  
          # creating a csv writer object
          csvwriter = csv.writer(csvfile, delimiter = ',')               
          csvwriter.writerows(result_list)                                   # Writing result list in csvfile

  except OSError as error:                          # Ensuring stat.csv file be over written every time script runs
      pass





In [ ]:
path = "/content/drive/MyDrive/HD Corpus/Files/"
statistics(path)

/content/drive/MyDrive/HD Corpus/Files/Phase 1/Output_files/hi_1385_output.txt {'Sp2', 'Sp1'}
3022
/content/drive/MyDrive/HD Corpus/Files/Phase 1/Output_files/hi_4370_output.txt {'Sp2', 'Sp1', 'Sp3'}
3111
/content/drive/MyDrive/HD Corpus/Files/Phase 1/Output_files/hi_6322_output.txt {'Sp2', 'Sp1', 'Sp3'}
3843
/content/drive/MyDrive/HD Corpus/Files/Phase 1/Output_files/hi_6435_output.txt {'Sp2', 'Sp1'}
5495
/content/drive/MyDrive/HD Corpus/Files/Phase 1/Output_files/hi_6441_output.txt {'Sp2', 'Sp1'}
3560
/content/drive/MyDrive/HD Corpus/Files/Phase 1/Output_files/hi_6523_output.txt {'Sp2', 'Sp1'}
7516
/content/drive/MyDrive/HD Corpus/Files/Phase 1/Output_files/hi_6579_output.txt {'Sp2', 'Sp1'}
3306
/content/drive/MyDrive/HD Corpus/Files/Phase 1/Output_files/hi_6585_output.txt {'Sp2', 'Sp1'}
5340
/content/drive/MyDrive/HD Corpus/Files/Phase 2/Output_files/hi_1580_output.txt {'Sp2', 'Sp1'}
6911
/content/drive/MyDrive/HD Corpus/Files/Phase 2/Output_files/hi_2142_output.txt {'Sp2', 'Sp1'}
7